---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
import re
import pprint
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    towns = open('university_towns.txt').read().split('\n')[:-1] 
    df = pd.DataFrame(columns = ["State", "RegionName"])
    for i, state in enumerate(towns):
        #state = re.sub(r'(\[[0-9]*\])*$', '', state)
        if state.split('[')[0].strip() in states.values() :
            for town in towns[i+1:]:
                town = re.sub(r'(\[[0-9]*\])*', '', town)
                if '[' in town:
                    break
                df = df.append(pd.Series({'State': state.split('[')[0].strip(), 'RegionName':town.split('(')[0].strip()})
                               , ignore_index = True)
                
            
    
    return df
    


get_list_of_university_towns()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy
6,Alabama,Tuscaloosa
7,Alabama,Tuskegee
8,Alaska,Fairbanks
9,Arizona,Flagstaff


In [4]:
def get_gpd():
    
    gpd = pd.read_excel('gdplev.xls', header=4)
    gpd = gpd.rename(columns = {'Unnamed: 0': 'year','Unnamed: 4':'quarter'})
    gpd = gpd[['quarter', 'GDP in billions of current dollars.1']].dropna()
    ind = gpd[gpd['quarter'].str.contains('2005')].index[0]
    gpd = gpd.loc[ind-1:]
    return gpd  


def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    gpd = get_gpd()
    
    col = 'GDP in billions of current dollars.1'
   
    for ind in gpd.index[1:]:
        if(gpd.loc[ind][col] <= gpd.loc[ind-1][col] and gpd.loc[ind+1][col] <= gpd.loc[ind][col]):
            return gpd.loc[ind-1]['quarter']
                       
        
    
    return gpd

get_recession_start()

'2008q3'

In [5]:
def get_recession_end():
    gpd = get_gpd()
    start = gpd[gpd['quarter'] == get_recession_start()].index[0] - gpd.index[0]
    col = 'GDP in billions of current dollars.1'
   
    for ind in gpd.index[start:-1]:
        if(gpd.loc[ind][col] >= gpd.loc[ind-1][col] and gpd.loc[ind+1][col] >= gpd.loc[ind][col]):
            return(gpd.loc[ind+1]['quarter'])
                       
    
    
    return gpd

get_recession_end()

'2009q4'

In [6]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    gpd = get_gpd()
    start = get_recession_start()
    end = get_recession_end()
    gpd = gpd.set_index('quarter')
    
    period = gpd[start:end]
    col = 'GDP in billions of current dollars.1'
    return np.argmin(period[col])

get_recession_bottom()

'2009q2'

In [7]:
from collections import defaultdict
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    cols = ['200{}q{}'.format(i,j+1) for i in range(17) for j in range(4) ]
    hos = pd.read_csv('City_Zhvi_AllHomes.csv')
    q1 = r'-0[1-3]$'
    q2 = r'-0[4-6]$'
    q3 = r'-0[7-9]$'
    q4 = r'-1[0-2]$'
    #return hos
    years = defaultdict(list)
    for col in hos.columns:
        if col[0] == '2':
            if re.search(q1,col) is not None:
                years[re.sub(q1,'q1',col)].append(col)
            elif re.search(q2,col) is not None:
                years[re.sub(q2,'q2',col)].append(col)
            elif re.search(q3,col) is not None:
                years[re.sub(q3,'q3',col)].append(col)
            elif re.search(q4,col) is not None:
                years[re.sub(q4,'q4',col)].append(col)
       
    
    #pprint.pprint(years)
    nhos = pd.DataFrame(columns = list(years.keys()))
    for newcol in years:
        nhos[newcol] = np.mean(hos[years[newcol]], axis = 1)
      
    
    nhos['State'] = hos['State'].replace(states)

    nhos['RegionName'] = hos['RegionName']
    nhos = nhos.set_index(["State","RegionName"])
    #print(type(nhos.loc["Texas"].loc["Austin"].loc["2010q3"]))
    return nhos



convert_housing_data_to_quarters()

,,2003q4,2016q2,2015q1,2001q4,2002q3,2013q1,2014q3,2001q2,2007q4,2006q2,...,2008q3,2016q1,2011q4,2012q1,2014q1,2009q4,2003q2,2010q1,2006q1,2004q4
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,5.916333e+05,5.322667e+05,NaN,NaN,4.690333e+05,5.228000e+05,NaN,5.133667e+05,5.137000e+05,...,4.997667e+05,5.828667e+05,4.637000e+05,4.602000e+05,5.031333e+05,4.583667e+05,NaN,4.697000e+05,4.881333e+05,4.321333e+05
California,Los Angeles,3.708333e+05,5.774667e+05,5.288000e+05,2.530333e+05,2.870000e+05,4.115333e+05,5.090667e+05,2.391000e+05,5.613667e+05,5.875000e+05,...,4.695000e+05,5.660333e+05,3.755667e+05,3.706000e+05,4.842667e+05,4.043333e+05,3.345000e+05,4.133667e+05,5.817667e+05,4.773667e+05
Illinois,Chicago,2.041333e+05,2.082000e+05,2.010667e+05,1.704333e+05,1.789667e+05,1.683667e+05,1.957667e+05,1.618000e+05,2.468333e+05,2.452000e+05,...,2.320000e+05,2.060667e+05,1.702000e+05,1.669333e+05,1.909000e+05,2.116667e+05,1.943000e+05,2.073000e+05,2.424000e+05,2.183333e+05
Pennsylvania,Philadelphia,7.073333e+04,1.269333e+05,1.162000e+05,5.753333e+04,6.216667e+04,1.120000e+05,1.153000e+05,5.553333e+04,1.208000e+05,1.123667e+05,...,1.169333e+05,1.234333e+05,1.147000e+05,1.127333e+05,1.137333e+05,1.185667e+05,6.620000e+04,1.211667e+05,1.083333e+05,8.553333e+04
Arizona,Phoenix,1.404333e+05,1.914333e+05,1.715333e+05,1.243000e+05,1.305333e+05,1.363333e+05,1.653667e+05,1.215667e+05,2.289333e+05,2.452667e+05,...,1.937667e+05,1.879000e+05,1.031667e+05,1.075000e+05,1.632333e+05,1.434667e+05,1.379333e+05,1.309333e+05,2.428000e+05,1.629333e+05
Nevada,Las Vegas,1.761667e+05,1.972000e+05,1.775000e+05,1.461333e+05,1.523333e+05,1.326333e+05,1.734000e+05,1.417333e+05,2.649333e+05,2.977333e+05,...,2.133667e+05,1.946000e+05,1.104000e+05,1.108667e+05,1.660000e+05,1.361333e+05,1.626667e+05,1.294667e+05,2.954333e+05,2.628000e+05
California,San Diego,4.120333e+05,5.362333e+05,5.033667e+05,2.919333e+05,3.311000e+05,4.164000e+05,4.890333e+05,2.762667e+05,4.811667e+05,5.383667e+05,...,4.246667e+05,5.293333e+05,3.665667e+05,3.697667e+05,4.740000e+05,3.985667e+05,3.774333e+05,4.061333e+05,5.415000e+05,5.251333e+05
Texas,Dallas,9.996667e+04,1.446000e+05,1.137000e+05,9.090000e+04,9.540000e+04,9.796667e+04,1.089000e+05,8.930000e+04,1.231333e+05,1.185333e+05,...,1.121667e+05,1.405000e+05,9.383333e+04,9.616667e+04,1.053333e+05,1.034333e+05,9.850000e+04,1.033000e+05,1.178667e+05,1.119000e+05
California,San Jose,4.871667e+05,8.189333e+05,7.314333e+05,4.455667e+05,4.732333e+05,5.727000e+05,6.970333e+05,4.702000e+05,6.714000e+05,6.855000e+05,...,5.831333e+05,8.036000e+05,4.882000e+05,4.861333e+05,6.611667e+05,5.065333e+05,4.737000e+05,5.148000e+05,6.851333e+05,5.767000e+05


In [8]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    hdf = convert_housing_data_to_quarters()
    start = get_recession_start()
    end = get_recession_end()
    bot = get_recession_bottom()
    hdf['PriceRatio'] = hdf[start].div(hdf[bot])
    
    utowns = get_list_of_university_towns()
    uni_list_of_tuples = [tuple(x) for x in utowns.values]
    uni_towns = hdf[hdf.index.isin(uni_list_of_tuples)]['PriceRatio'] 
#not sure why, I will update you when I know, this following doesnt always work
#non_uni_towns = hdf.loc[~uni_list_of_tuples]
    non_uni_towns = hdf[~hdf.index.isin(uni_list_of_tuples)]['PriceRatio'] 
    
    #print(np.mean(uni_towns) - np.mean(non_uni_towns))
    res = ttest_ind(uni_towns,non_uni_towns,nan_policy= 'omit')
    return (res[1] < 0.01, res[1],"university town" )

run_ttest()

(True, 0.0054964273536330264, 'university town')